In [73]:
import numpy as np
import pandas as pd
from collections import Counter

class CandleStick:
    def __init__(self, open, high, low, close):
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self._validate_candle()

    def __repr__(self):
        return f"Candle({self.open}, {self.high}, {self.low}, {self.close})"

    def __str__(self):
        return f"O: {self.open}, H: {self.high}, L: {self.low}, C: {self.close}"

    @property
    def open(self):
        return self._open

    @property
    def high(self):
        return self._high

    @property
    def low(self):
        return self._low

    @property
    def close(self):
        return self._close

    @open.setter
    def open(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("open must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("open must be positive")
        self._open = value

    @high.setter
    def high(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("high must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("high must be positive")
        self._high = value

    @low.setter
    def low(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("low must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("Low must be positive")
        self._low = value

    @close.setter
    def close(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError("close must be int or float not {}".format(type(value)))
        if value < 0:
            raise ValueError("close must be positive")
        self._close = value

    def _validate_candle(self):
        if self.open > self.high:
            raise ValueError("open cannot be greater than high")
        if self.open < self.low:
            raise ValueError("open cannot be less than low")
        if self.close > self.high:
            raise ValueError("close cannot be greater than high")
        if self.close < self.low:
            raise ValueError("close cannot be less than low")

    def type(self):
        if self.open < self.close:
            return 'bullish'
        elif self.open > self.close:
            return 'bearish'
        else:
            return 'doji'

    def cs_size(self):
        return abs(self.high - self.low)

    def upper_shadow_size(self):
        return self.high - max(self.open, self.close)

    def lower_shadow_size(self):
        return min(self.open, self.close) - self.low

    def body_size(self):
        return abs(self.close - self.open)

    def is_bullish(self):
        return self.type() == 'bullish'

    def is_bearish(self):
        return self.type() == 'bearish'

    def is_doji(self):
        return self.type() == 'doji'

    def cs_body_ratio(self):
        return self.body_size() / self.cs_size()

    def body_upper_shadow_ratio(self):
        return self.body_size() / self.upper_shadow_size() if self.upper_shadow_size() > 0 else 0

    def body_lower_shadow_ratio(self):
        return self.body_size() / self.lower_shadow_size() if self.lower_shadow_size() > 0 else 0

    def body_position(self):
        upper_shadow = self.high - max(self.open, self.close)
        lower_shadow = min(self.open, self.close) - self.low
        shadows = upper_shadow + lower_shadow
        return  (2 * lower_shadow) / shadows -1 if shadows != 0 else 0 # -1 = lower shadow, 0 = middle, 1 = upper shadow




############################################################################################################################################################





class CandleStickFrame:
    def __init__(self, open, high, low, close):
        open, high, low, close = self._validate_input(open, high, low, close)
        self.candle_sticks = [CandleStick(o, h, l, c) for o, h, l, c in zip(open, high, low, close)]
        self.df = pd.DataFrame({"open": open, "high": high, "low": low, "close": close})
        self._bullish_count, self._bearish_count, self._doji_count = self._type_count()

    def __repr__(self):
        return f"CandleFrame({self.df})"

    def __str__(self):
        return f"{self.df}"

    def __len__(self):
        return len(self.candle_sticks)

    def __getitem__(self, index):
        return self.candle_sticks[index]

    def __iter__(self):
        return iter(self.candle_sticks)

    def __reversed__(self):
        return reversed(self.candle_sticks)

    @staticmethod
    def _validate_input(open, high, low, close):
        if not isinstance(open, list) and not isinstance(open, np.ndarray) and not isinstance(open, pd.core.series.Series):
            raise TypeError("open must be list, np.ndarry, or pd.core.series.Series not {}".format(type(open)))
        if not isinstance(high, list) and not isinstance(high, np.ndarray) and not isinstance(high, pd.core.series.Series):
            raise TypeError("high must be list, np.ndarry, or pd.core.series.Series not {}".format(type(high)))
        if not isinstance(low, list) and not isinstance(low, np.ndarray) and not isinstance(low, pd.core.series.Series):
            raise TypeError("low must be list, np.ndarry, or pd.core.series.Series not {}".format(type(low)))
        if not isinstance(close, list) and not isinstance(close, np.ndarray) and not isinstance(close, pd.core.series.Series):
            raise TypeError("close must be list, np.ndarry, or pd.core.series.Series not {}".format(type(close)))
        open = list(open)
        high = list(high)
        low = list(low)
        close = list(close)
        if len(open) != len(high) or len(open) != len(low) or len(open) != len(close):
            raise ValueError("open, high, low, and close must be the same length")
        if not all(isinstance(x, (int, float)) for x in open):
            raise TypeError("open must be list of int or float not {}".format(type(open)))
        if not all(isinstance(x, (int, float)) for x in high):
            raise TypeError("high must be list of int or float not {}".format(type(high)))
        if not all(isinstance(x, (int, float)) for x in low):
            raise TypeError("low must be list of int or float not {}".format(type(low)))
        if not all(isinstance(x, (int, float)) for x in close):
            raise TypeError("close must be list of int or float not {}".format(type(close)))
        return open, high, low, close

    def _type_count(self):
        bullish = 0
        bearish = 0
        doji = 0
        for candle in self.candle_sticks:
            if candle.type() == 'bullish':
                bullish += 1
            elif candle.type() == 'bearish':
                bearish += 1
            elif candle.type() == 'doji':
                doji += 1
        return bullish, bearish, doji

    def _bullish_ratio(self):
        return self._bullish_count / len(self.candle_sticks)

    def _bearish_ratio(self):
        return self._bearish_count / len(self.candle_sticks)

    def _doji_ratio(self):
        return self._doji_count / len(self.candle_sticks)

    def type_ratio(self):
        return 'bullish: {:.2f}, bearish: {:.2f}, doji: {:.2f}'.format(self._bullish_ratio(), self._bearish_ratio(), self._doji_ratio())




############################################################################################################################################################





class CandleStickPattern:
    def __init__(self, candle_stick_frame):
        self.candle_stick_frame = candle_stick_frame
        self._average_candle_stick_size = self.average_candle_stick_size()
        self._average_body_size = self.average_body_size()

    def average_candle_stick_size(self):
        return sum([candle.cs_size() for candle in self.candle_stick_frame]) / len(self.candle_stick_frame)

    def average_body_size(self):
        return sum([candle.body_size() for candle in self.candle_stick_frame]) / len(self.candle_stick_frame)

    def trend(self, index, window=10):
        if len(self.candle_stick_frame) < window:
            raise ValueError("window must be less than or equal to the length of the candle stick frame")
        if window < 1:
            raise ValueError("window must be greater than 0")
        if index < 0:
            return None
        trend = 0
        for cs in self.candle_stick_frame[index-window:index]:
            if cs.type() == 'bullish':
                trend += cs.cs_size()
            elif cs.type() == 'bearish':
                trend -= cs.cs_size()
        if trend > 0:
            return 'up'
        elif trend < 0:
            return 'down'
        else:
            return None



    # Bullish Reversal Candlestick Patterns:

    # Hammer (1)
    def _is_hammer(self, trend, cs, cs_body_position = 0.25, body_ls_ratio = 0.35, body_us_ratio = 1.5):
        if trend is None:
            return None
        if trend == 'down':
            if cs.type() == 'bullish' and cs.body_position() > cs_body_position \
                    and cs.body_lower_shadow_ratio() < body_ls_ratio and cs.body_upper_shadow_ratio() < body_us_ratio:
                return True
        return False

    def is_hammer(self, trend_window=10, cs_body_position = 0.25, body_ls_ratio = 0.35, body_us_ratio = 1.5):
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            result.append(self._is_hammer(trend, self.candle_stick_frame[i], cs_body_position, body_ls_ratio, body_us_ratio))
        return result

    # Bullish Piercing (2)
    def _is_piercing(self, trend, cs, cs_m1):
        if trend is None:
            return None
        if trend == 'down':
            if cs.type() == 'bullish' and cs_m1.type() == 'bearish':
                if cs.open < cs_m1.close and cs_m1.open - cs_m1.body_size() / 2 < cs.close < cs_m1.open:
                    return True
        return False

    def is_piercing(self, trend_window=10):
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            result.append(self._is_piercing(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1]))
        return result

    # Bullish Engulfing (3)
    def _is_bullish_engulfing(self, trend, cs, cs_m1):
        if trend is None:
            return None
        if trend == 'down':
            if cs.type() == 'bullish' and cs_m1.type() == 'bearish':
                if cs.open < cs_m1.close and cs.close > cs_m1.open:
                    return True
        return False

    def is_bullish_engulfing(self, trend_window=10):
        result = []
        for i in range(len(self.candle_stick_frame)):
            cs = self.candle_stick_frame[i]
            cs_m1 = self.candle_stick_frame[i-1]
            trend = self.trend(i-1, trend_window)
            result.append(self._is_bullish_engulfing(trend, cs, cs_m1))
        return result

        # Morning Star (4)
    def _is_morning_star(self, trend, cs_m2, cs_m1, cs, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                         cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        if trend is None:
            return None
        elif trend == 'down':
            if cs_m2.is_bearish() and cs_m2.cs_body_ratio() >= cs_m2_body_ratio and cs_m2.cs_size() >= cs_m2_relative_size * self._average_candle_stick_size:
                if cs_m1.cs_size() <= cs_m1_relative_size * self._average_candle_stick_size:
                    if cs.is_bullish() and cs.cs_body_ratio() >= cs_body_ratio and cs.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                        return True
        return False

    def is_morning_star(self,trend_window=10, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                        cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-3, trend_window)
            cs_m2 = self.candle_stick_frame[i-2]
            cs_m1 = self.candle_stick_frame[i-1]
            cs = self.candle_stick_frame[i]
            results.append(self._is_morning_star(trend, cs_m2, cs_m1, cs, cs_m2_body_ratio, cs_body_ratio,
                                                 cs_m2_relative_size, cs_m1_relative_size, cs_relative_size))
        return results

    # Three White Soldiers (5)
    def _is_three_white_soldiers(self, trend, cs_m2, cs_m1, cs, cs_body_ratio=0.6, cs_relative_size=0.5):
        if trend is None:
            return None
        elif trend == 'down':
            if cs_m2.is_bullish() and cs_m2.cs_body_ratio() >= cs_body_ratio and cs_m2.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                if cs_m1.is_bullish() and cs_m1.cs_body_ratio() >= cs_body_ratio and cs_m1.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                    if cs.is_bullish() and cs.cs_body_ratio() >= cs_body_ratio and cs.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                        return True
        return False

    def is_three_white_soldiers(self, trend_window=10, cs_body_ratio=0.5, cs_relative_size=0.3):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-3, trend_window)
            cs_m2 = self.candle_stick_frame[i-2]
            cs_m1 = self.candle_stick_frame[i-1]
            cs = self.candle_stick_frame[i]
            results.append(self._is_three_white_soldiers(trend, cs_m2, cs_m1, cs, cs_body_ratio, cs_relative_size))
        return results

    # Bullish Marubozu (6)
    def _is_bullish_marubozu(self, trend, cs, cs_body_ratio=0.9):
        if trend is None:
            return None
        else:
            return cs.is_bullish() and cs.cs_body_ratio() >= cs_body_ratio and trend == 'down'

    def is_bullish_marubozu(self, trend_window=10, cs_body_ratio=0.9):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            cs = self.candle_stick_frame[i]
            results.append(self._is_bullish_marubozu(trend, cs, cs_body_ratio))
        return results


    # Bearish Reversal Candlestick Patterns:

    # Hanging Man (14)
    def _is_hanging_man(self, trend, cs, cs_body_position=0.25, body_ls_ratio=0.35, body_us_ratio=1.5):
        if trend is None:
            return None
        if trend == 'up':
            if cs.type() == 'bearish' and cs.body_position() > cs_body_position \
                    and cs.body_lower_shadow_ratio() < body_ls_ratio and cs.body_upper_shadow_ratio() < body_us_ratio:
                return True
        return False

    def is_hanging_man(self, trend_window=10, cs_body_position=0.25, body_ls_ratio=0.35, body_us_ratio=1.5):
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            result.append(self._is_hanging_man(trend, self.candle_stick_frame[i], cs_body_position, body_ls_ratio, body_us_ratio))
        return result

    # Dark Cloud (15)
    def _is_dark_cloud(self, trend, cs, cs_m1):
        if trend is None:
            return None
        if trend == 'up':
            if cs.type() == 'bearish' and cs_m1.type() == 'bullish':
                if cs.open > cs_m1.close and cs_m1.open + cs_m1.body_size() / 2 > cs.close > cs_m1.open:
                    return True
        return False

    def is_dark_cloud(self, trend_window=10):
        result = []
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            result.append(self._is_dark_cloud(trend, self.candle_stick_frame[i], self.candle_stick_frame[i-1]))
        return result

    # Bearish Engulfing (16)
    def _is_bearish_engulfing(self, trend, cs, cs_m1):
        if trend is None:
            return None
        if trend == 'up':
            if cs.type() == 'bearish' and cs_m1.type() == 'bullish':
                if cs.open > cs_m1.close and cs.close < cs_m1.open:
                    return True
        return False

    def is_bearish_engulfing(self, trend_window=10):
        result = []
        for i in range(len(self.candle_stick_frame)):
            cs = self.candle_stick_frame[i]
            cs_m1 = self.candle_stick_frame[i-1]
            trend = self.trend(i-1, trend_window)
            result.append(self._is_bearish_engulfing(trend, cs, cs_m1))
        return result

    # Evening Star (17)
    def _is_evening_star(self, trend, cs_m2, cs_m1, cs, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                         cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        if trend is None:
            return None
        elif trend == 'up':
            if cs_m2.is_bullish() and cs_m2.cs_body_ratio() >= cs_m2_body_ratio and cs_m2.cs_size() >= cs_m2_relative_size * self._average_candle_stick_size:
                if cs_m1.cs_size() <= cs_m1_relative_size * self._average_candle_stick_size:
                    if cs.is_bearish() and cs.cs_body_ratio() >= cs_body_ratio and cs.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                        return True
        return False

    def is_evening_star(self,trend_window=10, cs_m2_body_ratio=0.6, cs_body_ratio=0.6,
                        cs_m2_relative_size=0.6, cs_m1_relative_size=0.6, cs_relative_size=0.3):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-3, trend_window)
            cs_m2 = self.candle_stick_frame[i-2]
            cs_m1 = self.candle_stick_frame[i-1]
            cs = self.candle_stick_frame[i]
            results.append(self._is_evening_star(trend, cs_m2, cs_m1, cs, cs_m2_body_ratio, cs_body_ratio,
                                                 cs_m2_relative_size, cs_m1_relative_size, cs_relative_size))
        return results

    # Three Black Crows (18)
    def _is_three_black_crows(self, trend, cs_m2, cs_m1, cs, cs_body_ratio=0.6, cs_relative_size=0.5):
        if trend is None:
            return None
        elif trend == 'up':
            if cs_m2.is_bearish() and cs_m2.cs_body_ratio() >= cs_body_ratio and cs_m2.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                if cs_m1.is_bearish() and cs_m1.cs_body_ratio() >= cs_body_ratio and cs_m1.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                    if cs.is_bearish() and cs.cs_body_ratio() >= cs_body_ratio and cs.cs_size() >= cs_relative_size * self._average_candle_stick_size:
                        return True
        return False

    def is_three_black_crows(self, trend_window=10, cs_body_ratio=0.6, cs_relative_size=0.5):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-3, trend_window)
            cs_m2 = self.candle_stick_frame[i-2]
            cs_m1 = self.candle_stick_frame[i-1]
            cs = self.candle_stick_frame[i]
            results.append(self._is_three_black_crows(trend, cs_m2, cs_m1, cs, cs_body_ratio, cs_relative_size))
        return results

    # Bearish Marubozu (19)
    def _is_bearish_marubozu(self, trend, cs, cs_body_ratio=0.9):
        if trend is None:
            return None
        else:
            return cs.is_bearish() and cs.cs_body_ratio() >= cs_body_ratio and trend == 'up'

    def is_bearish_marubozu(self, trend_window=10, cs_body_ratio=0.9):
        results = list()
        for i in range(len(self.candle_stick_frame)):
            trend = self.trend(i-1, trend_window)
            cs = self.candle_stick_frame[i]
            results.append(self._is_bearish_marubozu(trend, cs, cs_body_ratio))
        return results

In [74]:
# load data
df = pd.read_csv('EURUSD_Daily.csv')
df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,2006.01.02,1.18490,1.18690,1.18010,1.18210,5870,0.000000e+00,20
1,2006.01.03,1.18220,1.20330,1.18100,1.20150,9455,0.000000e+00,20
2,2006.01.04,1.20140,1.21460,1.20120,1.21080,10123,0.000000e+00,20
3,2006.01.05,1.21070,1.21230,1.20650,1.20970,9584,0.000000e+00,20
4,2006.01.06,1.21010,1.21820,1.20780,1.21430,9219,0.000000e+00,20
...,...,...,...,...,...,...,...,...
4406,2022.12.23,1.05934,1.06323,1.05861,1.06150,762777,1.144170e+11,5
4407,2022.12.26,1.06196,1.06359,1.06140,1.06335,224659,3.369885e+10,5
4408,2022.12.27,1.06324,1.06689,1.06110,1.06385,757183,1.135770e+11,5
4409,2022.12.28,1.06385,1.06738,1.06058,1.06115,742676,1.114010e+11,5


In [75]:
# create candlestick frame
candle_stick_frame = CandleStickFrame(df['<OPEN>'], df['<HIGH>'], df['<LOW>'], df['<CLOSE>'])
len(candle_stick_frame)

4411

In [76]:
res = CandleStickPattern(candle_stick_frame).is_three_black_crows()
count = 0
for r in res:
    if r:
        count += 1
print(count)

10


In [78]:
candle_pattern = CandleStickPattern(candle_stick_frame)
df['is_bullish_marubozu'] = candle_pattern.is_bullish_marubozu()
df['is_bearish_marubozu'] = candle_pattern.is_bearish_marubozu()
df['is_morning_star'] = candle_pattern.is_morning_star()
df['is_evening_star'] = candle_pattern.is_evening_star()
df['is_bullish_engulfing'] = candle_pattern.is_bullish_engulfing()
df['is_bearish_engulfing'] = candle_pattern.is_bearish_engulfing()
df['is_hammer'] = candle_pattern.is_hammer()
df['is_hanging_man'] = candle_pattern.is_hanging_man()
df['is_piercing'] = candle_pattern.is_piercing()
df['is_dark_cloud'] = candle_pattern.is_dark_cloud()
df['is_three_white_soldiers'] = candle_pattern.is_three_white_soldiers()
df['is_three_black_crows'] = candle_pattern.is_three_black_crows()
df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,is_bullish_marubozu,is_bearish_marubozu,is_morning_star,is_evening_star,is_bullish_engulfing,is_bearish_engulfing,is_hammer,is_hanging_man,is_piercing,is_dark_cloud,is_three_white_soldiers,is_three_black_crows
0,2006.01.02,1.18490,1.18690,1.18010,1.18210,5870,0.000000e+00,20,None,None,None,None,None,None,None,None,None,None,None,None
1,2006.01.03,1.18220,1.20330,1.18100,1.20150,9455,0.000000e+00,20,None,None,None,None,None,None,None,None,None,None,None,None
2,2006.01.04,1.20140,1.21460,1.20120,1.21080,10123,0.000000e+00,20,None,None,None,None,None,None,None,None,None,None,None,None
3,2006.01.05,1.21070,1.21230,1.20650,1.20970,9584,0.000000e+00,20,None,None,None,None,None,None,None,None,None,None,None,None
4,2006.01.06,1.21010,1.21820,1.20780,1.21430,9219,0.000000e+00,20,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4406,2022.12.23,1.05934,1.06323,1.05861,1.06150,762777,1.144170e+11,5,False,False,False,False,False,False,False,False,False,False,False,False
4407,2022.12.26,1.06196,1.06359,1.06140,1.06335,224659,3.369885e+10,5,False,False,False,False,False,False,False,False,False,False,False,False
4408,2022.12.27,1.06324,1.06689,1.06110,1.06385,757183,1.135770e+11,5,False,False,False,False,False,False,False,False,False,False,False,False
4409,2022.12.28,1.06385,1.06738,1.06058,1.06115,742676,1.114010e+11,5,False,False,False,False,False,False,False,False,False,False,False,False


In [70]:
#save the df
df.to_csv('EURUSD_Daily_with_patterns.csv', index=False)